# Using Featuretools to analyze medals won at the Olympic Games

In this notebook, we will examine a dataset containing all the medals won by each athlete at each Summer Olympic Games. Our goal will be to see what quantities are important in predicting the future number of medals won.

To do this, we will build a machine-learning-based predictive model that is trained on historical data, and used to predict the number of medals won by each country in the next Olympics. We'll build this model using high-level transformations of the data &mdash; called features &mdash; that are automatically generated from Featuretools. [Featuretools](https://featuretools.com) is a Python library for [automated feature engineering]().

This notebook will serve as an introduction to Featuretools, but does not explain each function call in depth. See the [documentation](https://docs.featuretools.com) for a more thorough usage guide.

## Disclaimer on suitability of the dataset and problem

Ordinary machine learning with bare-bones features allows us
to predict the
quantities we care about in this example extremely
well. That means that advanced high-level features
from Featuretools do not produce much improvement.
However, they provide much more interpretable
predictive factors, which lets us examine which
ones were important and which ones we care about. In the other notebooks
in this repo, we will see how to use the more
advanced components of Featuretools to increase our
scores. Perhaps more importantly, we will also see how new feature types compare to each other in predictive poower in a relative way.

In [1]:
import pandas as pd
from sklearn.preprocessing import Imputer, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import confusion_matrix
import featuretools as ft
from featuretools import primitives as prims
from featuretools.selection.variance_selection import (
    select_high_variance_features,
    select_percent_null)
import os
from ml import (bin_labels,
                TimeSeriesSplitByDate,
                fit_and_score)
from load_entityset import load_entityset
from utils import (build_baseline_features,
                   get_feature_importances,
                   plot_confusion_matrix,
                   select_non_null_features,
                   select_multivalued_features)
from IPython.display import display
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ROOT_DATA_FOLDER = os.path.expanduser("~/olympic_games_data")

# Step 1: Load in data

`EntitySet` is the in-memory data structure Featuretools uses to build and calculate features. It essentially consists of:
 * a dictionary of [Pandas DataFrames](https://pandas.pydata.org/pandas-docs/stable/dsintro.html)
 * associated metadata on how they are linked
 * what semantic types they contain
 * how they vary in time.
 
For more details and tutorials, check out the [documentation](https://docs.featuretools.com/loading_data/using_entitysets.html).

To examine the following function in more detail, check out [load_entityset.py](load_entityset.py)

In [3]:
es = load_entityset(with_next_competing_info=True)
es

Entityset: Olympic Games
  Entities:
    disciplines (shape = [67, 3])
    countries (shape = [220, 8])
    olympic_games (shape = [27, 3])
    medals_won (shape = [11532, 7])
    sports (shape = [43, 2])
    ...And 3 more
  Relationships:
    medaling_athletes.Athlete -> athletes.Athlete
    medals_won.Country Olympic ID -> countries_at_olympic_games.Country Olympic ID
    countries_at_olympic_games.Olympic Games ID -> olympic_games.Olympic Games ID
    medals_won.Discipline -> disciplines.Discipline
    disciplines.Sport -> sports.Sport
    ...and 2 more

In [4]:
es['countries_at_olympic_games'].df.head()

,Country Olympic ID,Olympic Games Name,Olympic Games ID,Country,CompetedNextOlympics,Year
Country Olympic ID,,,,,,
0,0,Athens 1896-06-30,0,HUN,True,1896-06-30 00:00:00
1,1,Athens 1896-06-30,0,GRE,True,1896-06-30 00:00:00
2,2,Athens 1896-06-30,0,GER,True,1896-06-30 00:00:00
3,3,Athens 1896-06-30,0,ZZX,True,1896-06-30 00:00:00
4,4,Athens 1896-06-30,0,GBR,True,1896-06-30 00:00:00


## Load labels and cutoff times

Machine learning (in its [supervised](http://scikit-learn.org/stable/supervised_learning.html) flavor) is all about predicting a set of labels, given a set of training data. This is canonically represented by a *feature matrix*, whose rows are each numeric vectors of fixed length. Each row is associated with a single value, called a label. The machine learning algorithm is tasked with building a model that is good at predicting the label given the row vector.

In *predictive modeling*, the labels represent future
quantities. This makes the time associated with the
label crucial, since it might mean we can't use some
of the data. If we're trying to train our model to a
predict a label that happened last Tuesday, we shouldn't be
using the numbers from today.

In this particular case, our goal is to predict the number of medals won by each country in each subsequent Olympic Games, so our labels will be uniquely defined by a particular country and date of particular Olympic Games in which it competed.

[Coming Soon] Check out [GeneratingLabels.ipynb](GeneratingLabels.ipynb) for a walkthrough of how I generated these labels.

[Feature Labs](https://www.featurelabs.com/), the company that I work for and that maintains Featuretools, sells a platform that, among other things, makes this label generation process extremely easy. This process is known as [Prediction Engineering](https://www.featurelabs.com/resources/why.html).

In [5]:
label_file = os.path.join(ROOT_DATA_FOLDER, "num_medals_by_country_labels.csv")
label_df = pd.read_csv(label_file,
                       parse_dates=['Olympics Date'],
                       encoding='utf-8',
                       usecols=['Number of Medals', 'Olympics Date', 'Country'])
# Sort by the date of the Olympics, and by the country (to maintain a consistent ordering)
label_df.sort_values(['Olympics Date', 'Country'], inplace=True)
label_df.head()

,Number of Medals,Olympics Date,Country
8,2,1896-06-29 00:00:00,AUS
9,5,1896-06-29 00:00:00,AUT
5,6,1896-06-29 00:00:00,DEN
10,11,1896-06-29 00:00:00,FRA
4,7,1896-06-29 00:00:00,GBR


# Generate Features Using Deep Feature Synthesis


Featuretools automatically extracts high-level, interpretable features from EntitySets. This means we can exhaustively create many features, and use these to train our machine learning predictive model.

Moreover, it makes sure to calculate these features only using data on or before specified *cutoff times*. This is a subtle but immensely important point &mdash; I'll refer you to the [documentation](https://docs.featuretools.com/automated_feature_engineering/handling_time.html) for deeper understanding. In this case, these cutoff times will be dates immediately prior to each Olympic Games. They are specified in the label file.

In [6]:
# Just want the Country and time to compute features (we're removing the label column here)
cutoff_times = label_df[['Country', 'Olympics Date']]
# Code is the index of the "countries" entity in the entityset (short for Country Code)
cutoff_times = cutoff_times.rename(columns={'Country': 'Code'})

We can specify the particular primitives we want Featuretools to use to construct features. Featuretools will walk through our EntitySet and apply these primitives recursively using the [Deep Feature Synthesis](https://docs.featuretools.com/automated_feature_engineering/afe.html) algorithm. These primitives are basic functions that take a particular data type as input (such as a Numeric or Categorical), and output a single value per *instance* (row) of an entity.

Transform primitives take each instance of an entity and apply a function to each row.

Aggregation primitives take 2 entities, where each row in the *parent* entity is connected to many rows in the *child*, and apply a function to all the rows in the *child* connected to a single *parent* row.


These primitives are defined in more detail [here](https://docs.featuretools.com/automated_feature_engineering/primitives.html)

In [7]:
agg_primitives = [
    prims.Sum, prims.Std, prims.Max, prims.Min, prims.Mean, prims.Count,
    prims.PercentTrue, prims.NUnique, prims.Mode, prims.Trend, prims.Skew
]
trans_primitives = [
    prims.Percentile
]

# Comparison to a Baseline

We want to make sure we use the features we used for our simple baseline model for a fair comparison to our more interpretable Featuretools model. To do this, we define these features manually using Featuretools, and add them in as *seed features* to DFS. This will not only include these features directly, but will also build higher-level features on top of them.

For a more thorough walkthrough of the baseline, check out [BaselineSolutions.ipynb](BaselineSolutions.ipynb).

In [ ]:
# Include our baseline predictors as seed features
num_medals_each_olympics, mean_num_medals = build_baseline_features(es)

## Run DFS

A note on the parameter settings:
  * `target_entity` defines the entity containing unique rows (called *instances*) for each training example. 
  * `trans_primitives` is a list of *transform* primitive classes.
  * `agg_primitives` is a list of *aggregation* primitive classes.
  * `max_depth` defines how *deep* we build features on top of one another ([explanation](https://docs.featuretools.com/automated_feature_engineering/afe.html#creating-deep-features))
  * `cutoff_times` is a Pandas DataFrame with a column for defining each instance we want to compute features for, and its associated cutoff time.

In [ ]:
feature_matrix, features = ft.dfs(
    entityset=es,
    target_entity="countries",
    trans_primitives=trans_primitives,
    agg_primitives=agg_primitives,
    max_depth=4,
    seed_features=num_medals_each_olympics + [mean_num_medals],
    cutoff_time=cutoff_times,
    verbose=True)
len(features)

Building features: 2847it [00:03, 671.65it/s] 
Progress:   4%|▎         | 1/27 [00:02<01:11,  2.75s/cutoff time]

# Remove features with zero variance or almost all null

Featuretools contains a couple handy functions for removing features with all null values or zero variance.

In [ ]:
feature_matrix, features = select_non_null_features(feature_matrix, features)

print "%d features selected" % len(features)

feature_matrix, features = select_multivalued_features(feature_matrix, features)

print "%d features selected" % len(features)

# One-hot-encode categorical features

Machine learning algorithms require all numeric values as input, but Featuretools by default produces some categorical values. One way to transform these values into numbers is to create several binary features that encode whether the feature is equal to each particular category (by default capped at the top 10 most common categories). To learn more check out this [docs page](https://docs.featuretools.com/guides/tuning_dfs.html#encoding-categorical-features).

In [ ]:
feature_matrix_encoded, features_encoded = ft.encode_features(feature_matrix, features)

# Binarize and bin labels

We'll actually create 3 slightly different models:

  1. **Regression** Predicting the actual number of medals per country
  2. **Binary Classification** Predicting whether the number of medals is greater than 10.
  3. **Binned Classification** Predicting which bin the medals falls into (i.e. between 0 and 2, 2 and 6, 6 and 10, 10 and 50, or greater than 50).

In [ ]:
dates = label_df['Olympics Date']
labels = label_df['Number of Medals']
binned_labels, bins = bin_labels(labels, [2, 6, 10, 50])
binary_labels = (labels >= 10).astype(int)

In [ ]:
binned_labels.value_counts()

In [ ]:
binary_labels.value_counts()

## Separate out the baseline features

These are for the slightly more complex, machine-learning-based "Baseline 2" from [BaselineSolutions.ipynb](BaselineSolutions.ipynb). Since the other baseline is not machine-learning-based, refer to that notebook for scores and analysis (they are significantly worse).

In [ ]:
just_baseline = feature_matrix_encoded[[
    f.get_name() for f in num_medals_each_olympics
    if f.get_name() in feature_matrix_encoded
]]
just_baseline.iloc[-10:, -10:]

# Build machine learning models and generate scores

## Create scikit-learn estimators

We'll use stock machine learning algorithms from scikit-learn, as well as an imputer that replaces missing values with the mean over that feature, and a scaler that makes sure each feature's values vary from 0 to 1. The `RobustScaler` class is more sensitive to outliers than the `StandardScaler`.

In [ ]:
pipeline_preprocessing = [("imputer",
                           Imputer(missing_values='NaN', strategy="mean", axis=0)),
                          ("scaler", RobustScaler(with_centering=True))]
rf_reg = RandomForestRegressor(n_estimators=100, n_jobs=-1)
rf_clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

# Time-based cross-validation

The way we'll do cross-validation is to separate training sets for each Olympics since 1960, using all historical data in the past for each.

In [ ]:
splitter = TimeSeriesSplitByDate(dates=dates, earliest_date=pd.Timestamp('1/1/1960'))

In [ ]:
X = feature_matrix_encoded.values
y = labels.values
y_binary = binary_labels.values
y_binned = binned_labels.values
X_baseline = just_baseline.values

### Modeling

`fit_and_score` takes in a feature matrix, labels, splitter, and the machine learning pipeline, builds models for each split, and scores each one using various metrics.

In [ ]:
reg_pipeline = Pipeline(pipeline_preprocessing + [("rf_reg", rf_reg)])
scores = fit_and_score(X, y, splitter, reg_pipeline, _type='regression')
baseline_scores = fit_and_score(X_baseline, y, splitter, reg_pipeline, _type='regression')
print "Regression model"
print "  R2 mean score:  %.2f +/- %.2f" % (scores['r2'].mean(),
                                           scores['r2'].std())
print "  Baseline R2 mean score:  %.2f +/- %.2f" % (baseline_scores['r2'].mean(),
                                                    baseline_scores['r2'].std())
print "  MSE mean score:  %.2f +/- %.2f" % (scores['mse'].mean(),
                                            scores['mse'].std())
print "  Baseline MSE mean score:  %.2f +/- %.2f" % (baseline_scores['mse'].mean(),
                                                         baseline_scores['mse'].std())

In [ ]:
pipeline = Pipeline(pipeline_preprocessing + [('rf_clf', rf_clf)])
binary_scores = fit_and_score(X, y_binary, splitter, pipeline, _type='classification')
baseline_binary_scores = fit_and_score(X_baseline, y_binary, splitter, pipeline, _type='classification')
binned_scores = fit_and_score(X, y_binned, splitter, pipeline, _type='classification')
baseline_binned_scores = fit_and_score(X_baseline, y_binned, splitter, pipeline, _type='classification')
print "Classification model"
print "  AUC mean score:  %.2f +/- %.2f" % (binary_scores['roc_auc'].mean(),
                                            binary_scores['roc_auc'].std())
print "  Baseline AUC mean score:  %.2f +/- %.2f" % (baseline_binary_scores['roc_auc'].mean(),
                                                     baseline_binary_scores['roc_auc'].std())
print "  F1 mean score:  %.2f +/- %.2f" % (binary_scores['f1'].mean(),
                                           binary_scores['f1'].std())
print "  Baseline F1 mean score:  %.2f +/- %.2f" % (baseline_binary_scores['f1'].mean(),
                                                    baseline_binary_scores['f1'].std())
print "  Binned F1 (micro averaged) mean score:  %.2f +/- %.2f" % (binned_scores['f1_micro'].mean(),
                                                                   binned_scores['f1_micro'].std())
print "  Baseline Binned F1 (micro averaged) mean score:  %.2f +/- %.2f" % (baseline_binned_scores['f1_micro'].mean(),
                                                                            baseline_binned_scores['f1_micro'].std())

# Comparing scores across years

We'll plot the scores over time for each type of problem, and then look at a few confusion matrices for particular years.

### Plotting the regression scores

In [ ]:
scores.set_index('Olympics Year')['r2'].plot()

### Plotting the binned scores


In [ ]:
binned_scores.set_index('Olympics Year')['f1_micro'].plot()

### Plotting the binary scores

In [ ]:
binary_scores.set_index('Olympics Year')['f1'].plot()

Notice how the binned problem is harder in different years than the regression problem. 1984 is a terrible year for the binned predictor, while 1992 is much worse for the regression predictor. The binary score is on the whole smoother over time, but still has a major hiccup in 1984.

In [ ]:
binary_scores

## Confusion Matrices

The confusion matrix is a good way to quickly see well a model is predicting binary outcomes. For an intro to what these are, see Scikit-Learn's [User Guide](http://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix).
Let's plot the confusion matrix for 1984 (a hard-to-predict year) and 2004 (an easy-to-predict year) (using code taken from Scikit-Learn [here](http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py))

In [ ]:
# 1984 = 5th split
split, year = 5, '1984'
train, test = splitter.split(X, y_binary)[split]
pipeline.fit(X[train], y_binary[train])
y_pred = pipeline.predict(X[test])
cm = confusion_matrix(y_binary[test], y_pred)
plot_confusion_matrix(cm, ['Won < 10 Medals', 'Won >= 10 Medals'], title=year)

In [ ]:
# 2004 = 10th split
split, year = 10, '2004'
train, test = splitter.split(X, y_binary)[split]
pipeline.fit(X[train], y_binary[train])
y_pred = pipeline.predict(X[test])
cm = confusion_matrix(y_binary[test], y_pred)
plot_confusion_matrix(cm, ['Won < 10 Medals', 'Won >= 10 Medals'], title=year)

In 1984, we underestimated how many medals 8 countries scored, while in 2004, we underestimated only 2 countres and overestimated 2. As we'll see later, this is probably due to the boycott in 1984 by Soviet Bloc countries, leading to higher medals scored by the democratic nations like the USA.

## Comparison to baseline

As mentioned in the disclaimer at the top, we actually do almost exactly the same as our baseline. The reason for this is most likely two-fold:
1. This is in general a very easy problem.
2. For particular cases this is a very hard problem for which additional data is needed to make better predictions.

We will examine the second issue in a future notebook. The problem arises from the fact that many outside factors contribute to weird outcomes at the Olympics which don't exist in our dataset. Boycotts and steroid scandals are a key example. Using only this current dataset detailing each medal that was won, there is no way for us to know which countries refrained from competing or which were disqualified after the fact.

However, our interpretable features now provide us with a way to examine which hidden factors are predictive of the outcome. The baseline features are essentially uninterpretable- they are simply the number of medals scored at each prior Olympic Games.

# Examine feature importances

This utility function grabs the Scikit-Learn Random Forest's built in feature importances, sorts them, and groups them by time split. These refer to how much weight the algorithm placed on each feature in order to predict the labels. 

We'll print out the top 10 highest weighted features for each Olympics since 1960.

In [ ]:
feature_imps_over_time = get_feature_importances(pipeline,
                                                 feature_matrix_encoded,
                                                 binary_labels,
                                                 splitter,
                                                 n=400)

In [ ]:
for test_year in [1964, 2012]:
    print "Test year: %s" % test_year
    test_date = pd.Timestamp('6/29/{}'.format(test_year))
    display(feature_imps_over_time[test_date].iloc[:10].reset_index(drop=True))

We see that features built on Baseline 1 ("mean_num_medals") are pretty high up in the list, but we do some interesting transformations on them. In particular, the Percentile feature is represented across the board.

We will examine particular features and what they mean in more depth later on.

# Supervised Feature Selection

One interesting workflow that Featuretools allows is the ability to select a subset of features from an exhaustive list based on which ones the RandomForest placed a lot of weight on.

To do this, let's rerun the computation using just the top 400 features

In [ ]:
top400 = feature_imps_over_time[pd.Timestamp('6/29/2012')]['Feature'].tolist()
important_feature_matrix = feature_matrix_encoded[top400]

pipeline = Pipeline(pipeline_preprocessing + [('rf_clf', rf_clf)])
binned_scores = fit_and_score(important_feature_matrix.values, y_binned, splitter, pipeline, _type='classification')
print "  F1 micro mean score:  %.2f +/- %.2f" % (binned_scores['f1_micro'].mean(),
                                                 binned_scores['f1_micro'].std())

# Comparison to full feature set

We do approximately the same as our original model. What's more interesting is that we now have a smaller set of highly explainable features. We can reduce this feature set even more without a drop in accuracy. Here it is with 50 features.

In [ ]:
important_feature_matrix = feature_matrix[top400[:50]]
pipeline = Pipeline(pipeline_preprocessing + [('rf_clf', rf_clf)])
binned_scores = fit_and_score(important_feature_matrix.values, y_binned, splitter, pipeline, _type='classification')
print "  F1 micro mean score:  %.2f +/- %.2f" % (binned_scores['f1_micro'].mean(),
                                                 binned_scores['f1_micro'].std())

## Smaller set of important features

Now let's take a closer look at the highest weighted features

In [ ]:
feature_imps_over_time = get_feature_importances(pipeline,
                                                 important_feature_matrix,
                                                 binary_labels,
                                                 splitter,
                                                 n=50)

for test_year in [1964, 1976, 2012]:
    print "Test year: %s" % test_year
    test_date = pd.Timestamp('6/29/{}'.format(test_year))
    display(feature_imps_over_time[test_date].iloc[:10].reset_index(drop=True))

Now we see some interesting thins.

For one, the important features make a lot of sense. For 1964, the most important feature is `PERCENTILE(COUNT(medals_won WHERE Medal = Gold))`. This refers to the percentile across all countries of the count of gold medals won in all previous Olympics.

The top feature in 1976 is more complicated but makes sense once we parse it. It's `MEAN(countries_at_olympic_games.PERCENTILE(NUM_UNIQUE(medals_won.MODE(medaling_athletes.Athlete))))`. This is a convoluted way of saying take the number of unique medaling athletes at each olympics a country competed in, find the percentile of that number in relation to the other countries, and take the average percentile over time. In other words, the average rank of the number of medaling athletes over time. The Mode part is an overcomplication that does not provide any additional information. So, if this value was high, it means that the country had on average a lot of medaling athletes in its historical Olympic showings.

In 2012, a few features down we have `TREND(countries_at_olympic_games.SKEW(medals_won.NUM_UNIQUE(medaling_athletes.Athlete)), Year)`. The way we have out data set up, each medal can be associated with many athletes (think: all the members of the medaling Soccer team). Therefore, this feature is looking at the skew (a statistical measure indicating whether more of the values lie toward the left or the right of the mean) of how many athletes there were per medal, and finding the the slope of a trend line fitted these skew numbers over time. One way to think of the skew number is whether the country won more medals in team sports than in individual sports, and the trend shows how that changes over time.

## Why were some years difficult to predict?
Specifically, the years between 1970 and 1992 showed high variability in scores.

The Olympics in the 1980s were marred by controversies. Summarizing the
[Wikipedia article](https://en.wikipedia.org/wiki/List_of_Olympic_Games_scandals_and_controversies):
1. Rule changes
   - This is the period that the IOC gradually started allowing professional
   athletes
2. Geopolitics
   - Widespread boycotts due to the Cold War
   - Montreal 1976: 22 African nations boycotted (to protest New Zealand's
   rugby team's tour of South Africa during Apartheid). China and Taiwan both
   boycotted. Fewest number of countries (92) participating since 1960.
   - Moscow 1980: US boycotts because of Soviet invasion of Afghanistan.
      Even fewer number of countries (80) than 1976
   - Los Angeles 1984: Soviet Union and 15 Eastern Bloc nations boycott
   because of "safety concerns". (However, a record 140 National Olympic
   Committees took part)
      Even fewer number of countries (80) than 1976
3. Performance-enhancing Drugs
   - late 1970s and especially 1980s ushered in the era of steroids
   - quote from 1989 Australian study "There is hardly a medal winner at the
   Moscow Games, certainly not a gold medal winner, who is not on one sort of
   drug or another: usually several kinds. The Moscow Games might as well have
   been called the Chemists' Games." See the wiki
   [page](https://en.wikipedia.org/wiki/Olympic_Games#Use_of_performance-enhancing_drugs), citation 183.
4. Gender discrimination
   - Many countries didn't send women until the late 1990s

[Coming Soon] For a more in-depth analysis, check out [AdvancedFeaturetools.ipynb](AdvancedFeaturetools.ipynb) and [LinkingDatasets.ipynb](LinkingDatasets.ipynb)

# Let's make a prediction about the 2016 olympics

We've trained models to predict the highest medaling countries in past Olympic Games, so we might as well use those models to predict who will receive the most medals in the next Olympics. This dataset came out before 2016, so we can compare to what actually happened. Unfortunately, the way our model is set up we would have to add in the 2016 data to make predictions about 2020 (I'll leave that up to you to try out yourself).

In [ ]:
# Train on all data
reg_pipeline.fit(important_feature_matrix, y)

# Get the top feature objects to compute
important_features = {f.get_name(): f for f in features_encoded
                      if f.get_name() in important_feature_matrix.columns}.values()
# Compute feature matrix on all data (no cutoff time)
# Leave cutoff_date blank to use the current time
df = es['countries_at_olympic_games'].df
countries_2012 = df.loc[df['Year'] == pd.Timestamp('2012-06-30'), 'Country']
latest_fm = ft.calculate_feature_matrix(features=important_features,
                                        instance_ids=countries_2012,
                                        cutoff_time=None,
                                        verbose=True)
# Make sure order is correct
latest_fm = latest_fm[important_feature_matrix.columns]

predictions = reg_pipeline.predict(latest_fm)
predictions_by_country = pd.Series(predictions, index=latest_fm.index)
predictions_by_country.sort_values(ascending=False, inplace=True)
predictions_by_country.head(20).plot(kind='bar')

The actual medal count is available [here](https://en.wikipedia.org/wiki/2016_Summer_Olympics_medal_table).
I took the first 20 to plot:

In [ ]:
actual_2016 = pd.DataFrame.from_records([('USA', 121),
('CHN', 70) ,
('GBR', 67) ,
('RUS', 56) ,
('GER', 42) ,
('FRA', 42) ,
('JPN', 41) ,
('AUS', 29) ,
('ITA', 28) ,
('CAN', 22) ,
('KOR', 21) ,
('NED', 19) ,
('BRA', 19) ,
('NZL', 18) ,
('KAZ', 18) ,
('AZE', 18) ,
('ESP', 17) ,
('HUN', 15) ,
('DEN', 15) ,
('KEN', 1)], columns=['Country', 'Medals']).set_index(['Country'])['Medals']

actual_2016.plot(kind='bar')

Looks like we overestimated Russia and underestimated Great Britain and China!

However- recall that in this past Olympics Russia was implicated in a major state-sponsored doping scandal. This resulted in the disqualification of essentially the entire track & field team. So Russia would otherwise probably have score many more medals.

# A fun prediction
Our data includes all the countries that have ever medaled in any Olympics. There's nothing stopping us from generating predictions for defunct countries. Let's see what happens if we predict the number of medals won in 2016 for every historical country.

In [ ]:
# Leave instance_ids blank to use all
latest_fm = ft.calculate_feature_matrix(features=important_features,
                                        cutoff_time=None,
                                        verbose=True)
# Make sure order is correct
latest_fm = latest_fm[important_feature_matrix.columns]

predictions = reg_pipeline.predict(latest_fm)
predictions_by_country = pd.Series(predictions, index=latest_fm.index)
predictions_by_country.sort_values(ascending=False, inplace=True)
predictions_by_country.head(20).plot(kind='bar')

Our model thinks that if the former Soviet Union were alive today it would win it all, the unified post-Soviet breakup team (EUN) would come in 3rd, and both East Germany (GDR) and West Germany (FRG) would come out ahead of modern-day Germany (GER).